In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
### IMPORTS
import pandas as pd
import os
from pprint import pprint

# Data

In [ ]:
# Synthetic Constants

DATA = "data/demo.csv"
OUTCOME = "Y"
NUM_TRAIN = 800

In [ ]:
# Diabetes Constants

DATA = "data/diabetes_binary.csv"
OUTCOME = "Outcome"
NUM_TRAIN = 600

### Option 1: Synthetic Data

Generates synthetic data according to a series of functions defined in generating_functions that basically make up a function graph. 

Data Descriptions:

- demo: Generated with function graph based on the DAG in figure TODO in paper (used for demoing purposes)
- standard: Generated with function graph based on the DAG in figure TODO in paper
- upstream_shift: Generated with function graph based on the DAG in figure TODO in paper with a significant change to one of the generating functions of a variables that comes before Y in a topological ordering to be used for distribution shift testing
- downstream_shift: Generated with function graph based on the DAG in figure TODO in paper with a significant change to one of the generating functions of a variables that comes after Y in a topological ordering to be used for distribution shift testing

- mixed_standard: Generated with function graph based on the ADMG in figure TODO in paper




In [ ]:
### GENERATE DATA

from generatedata import generate

# generating_functions contants several function nets to generate random data
from generating_functions import demo, standard, mixed_standard, upstream_shift, downstream_shift


generating_data = demo

starting_names = generating_data["starting_names"]
starting_generating_boundaries = generating_data["starting_generating_boundaries"] 
downstream_names = generating_data["downstream_names"]
downstream_generating_functions = generating_data["downstream_generating_functions"]
downstream_parents = generating_data["downstream_parents"]
data = generate(starting_names, starting_generating_boundaries, downstream_names, downstream_generating_functions, downstream_parents, 1000)

df = pd.DataFrame(data)
df.to_csv(generating_data["name"], index=False)


In [ ]:
### BINARIZE DATA

from binarize import binarize

binarize(generating_data["name"], generating_data["name"])


### Option 2: Real Data

You can import real data in as a csv file and run the same analysis. This software can only handle binary variables so most real world data will need to be cleaned first.

The cell will clean the data so that it can run in the software but you may lose more information that you want. It removes all rows with non-numeric typed data and then binarizes everything that is left.


In [ ]:
### CLEAN DATA
# Only need this if using real data

from fix import remove_strs

remove_strs(DATA)
binarize(DATA, DATA)

### Run the following cell to load whatever data you are using

The data MUST be binary

In [ ]:
### READ DATA

df = pd.read_csv(DATA)
df.head()

# Making the Causal Graph

### Option 1: Dag Drawing Software

pass and nodes and edges that you want to be preloaded into the software by changing the nodes and edges lists below

Some Useful commands when adding nodes:

- click: adds node
- shift+click: moves nearby node to second click (node to move will turn green)
- m: Your next click will select a nearby node to be moved and the following will place it
- Esc: cancels current action

Some Useful commands when adding Edges:

- click 2x: first click starts edge from nearby node and second click connects edge to second node. Note first click is a parent of second click (parent node will turn red)
- Esc: cancels current edge


General Useful commands:

- Cmd-z: Undo
- Cmd-shift-z: Redo
- t: toggle between adding nodes and edges
- p: Returns nodes and edges 


Currently only works for DAGs but you can still print out the directed edges for a ADMG, just make sure to change it from saying "edges" to "di_edges" and then enter the bi_edges in the manual entry phase.


In [ ]:
### DAG DRAWING SOFTWARE
# ADMG drawer coming soon

nodes = list(df.columns)
edges = []

_ = os.system(f"python run_dag_draw.py \"{str(nodes)}\" \"{str(edges)}\"")

In [ ]:
### COPY AND RUN OUTPUT OF ABOVE CELL HERE


### Option 2: Causal Discovery

If you think there is unmeasured confounding in your data run the ADMG discovery cell, otherwise run the DAG discovery. 

You can enter any prior knowledge using the box below

This software requires all edges to be oriented, to do that run the FIX DAG /ADMG cell to finish orienting edges with user knowledge. 

Note: to quit the edge fixing protocol enter q

Important: This is NOT guaranteed to not generate new cycles so be cautions with how edges are oriented.

DAG Edges:

Blue edges of form X --> Y means X causes Y
Brown edges of form X --- Y reprisent edges that could either be oriented X --> Y or Y --> X.

ADMG Edges:

Blue edges of form X --> Y means X causes Y
Red edges of form X <-> Y reprisent unmeasured confounding between X and Y
Green edges of form X --> Y really should be of form X o-> Y. This means either X --> Y or X <-> Y or both.
Orange edges of form X --- Y really should be of form X o-o Y. This means either 1) X --> Y, 2) Y --> X, 3) X <-> Y, 1 and 3, or 2 and 3

You can also add prior knowledge to the graph before running a search in the form of tiers. To do this create a file containing the knowlege then updating the knowledge=None to be knowledge=\<filename\>. A knowledge file is formatted as follows:


[tier num] nodes in tier


[tier num] nodes in tier


Example:

1 Age

2 Pregnancies DiabetesPedigreeFunction Outcome BMI SkinThickness BloodPressure Insulin Glucose




In [ ]:
# DAG DISCOVERY
from discovery import run_pc, draw

nodes, edges = run_pc(DATA, knowledge=None)

draw(nodes, edges)

In [ ]:
# ADMG DISCOVERY
from discovery import run_fci, draw

nodes, edges = run_fci(DATA)#, knowledge="knowledge.txt")

draw(nodes, edges)



In [ ]:
### FIX GRAPHS

from fix import fix_graph

graph_data = fix_graph(nodes, edges)

if len(graph_data) == 3:
    nodes, di_edges, bi_edges = graph_data
    to_draw = (nodes, di_edges, bi_edges)

    
else: 
    nodes, edges = graph_data
    to_draw = nodes, edges
    
draw(*to_draw)

### Option 3: Manual Entry

Here you can manually enter nodes and edges if you want instead. 

I also provide 3 graph dictionaries that contain prestored data.
- standard_synthetic: this is the DAG outlined in figure TODO of the paper
- mixed_synthetic: this is the ADMG outlined in figure TODO of the paper
- diabetes: this is a DAG of the diabetes dataset that was complied as a combination of PC algorithm search and outside knowledge.


In [ ]:
### MANUAL ENTRY
# Enter nodes and edges here
# Note: do not run if using option 1 or 2 because this will overwrite process
from graphs import standard_synthetic, mixed_synthetic, diabetes



nodes = standard_synthetic["nodes"]

# DAG
edges = standard_synthetic["edges"]

# ADMG
di_edges = mixed_synthetic["di_edges"]
bi_edges = mixed_synthetic["bi_edges"]


### Graph Dictionary

The data about a graph is held in a graph dictionary for ease of use

In [ ]:
# FOR DAG

current_graph = {
    "data": DATA,
    "outcome": OUTCOME,
    "nodes": nodes,
    "edges": edges
}

In [ ]:
# FOR ADMG

current_graph = {
    "data": DATA,
    "outcome": OUTCOME,
    "nodes": nodes,
    "di_edges": di_edges,
    "bi_edges": bi_edges,
}

# Scoring Causal Features

### Option 1: Using Augmented IPW, Frontdoor IPW, and IV Adjustment

(There is no other option)

Systematically calculate the causal effect of each node on the outcome.

For ADMGs if a valid adjusment set cannot be found then it attempts frontdoor then IV.

If there is no causal effect the effect is set to -1.0, if the effect cannot be computed using Augmented IPW, Frontdoor IPW, or IV Adjustment then the effect is set to 0.0 because it was unable to be determined. 

The scores output below are in order of most causally predictive of the outcome and thus should be ideal for training a ML model. 

In [ ]:
### USE THIS FOR DAGs

from findsubset import calculate_causal_scores_DAG

scores = calculate_causal_scores_DAG(df, nodes, edges, OUTCOME)

pprint(scores)

In [ ]:
### USE THIS FOR ADMGs

from findsubset import calculate_causal_scores_ADMG

scores = calculate_causal_scores_ADMG(df, nodes, di_edges, bi_edges, OUTCOME)

pprint(scores)


# Machine Learning

I implemented several different machine learning algorithms to use along with the feature selection.

During testing, all models perform significantly better when using an ideal subset (determined causally) compared to a random one.

Models Implemented:

- Logistic Regression
- Decision Trees
- Boosted Decision Trees
- Bagged Decision Trees
- Random Forrests
- Feedforward Neural Network



In [ ]:
SUBSET_SIZE = 3

sorted_causers = list(scores.keys())
ideal_subset = sorted_causers[:SUBSET_SIZE]

In [ ]:
from learning import prepare_data, LogisticRegression, DecisionTree, BaggedDecisionTree, BoostedDecisionTree, RandomForrest, NeuralNetwork
feats, Xtrain, Ytrain, Xtest, Ytest = prepare_data(DATA, NUM_TRAIN, OUTCOME, offset=True, n=SUBSET_SIZE, subset=ideal_subset)


In [ ]:
# Logistic Regression

model = LogisticRegression(len(feats))
model.fit(Xtrain, Ytrain)
print("Accuracy:", model.evaluate(Xtest, Ytest))


In [ ]:
# No longer want the offset term
feats, Xtrain, Ytrain, Xtest, Ytest = prepare_data(DATA, NUM_TRAIN, OUTCOME, offset=False, n=SUBSET_SIZE, subset=ideal_subset)

In [ ]:
# Decision Tree

model = DecisionTree()
model.fit(Xtrain, Ytrain)
print("Accuracy:", model.evaluate(Xtest, Ytest))


In [ ]:
# Boosted Decision Tree

model = BoostedDecisionTree()
model.fit(Xtrain, Ytrain)
print("Accuracy:", model.evaluate(Xtest, Ytest))


In [ ]:
# Bagged Decision Tree

model = BaggedDecisionTree()
model.fit(Xtrain, Ytrain)
print("Accuracy:", model.evaluate(Xtest, Ytest))


In [ ]:
# Random Forrest

model = RandomForrest()
model.fit(Xtrain, Ytrain)
print("Accuracy:", model.evaluate(Xtest, Ytest))


In [ ]:
# Neural Network
model = NeuralNetwork(len(feats))
model.fit(Xtrain, Ytrain)
print("Accuracy:", model.evaluate(Xtest, Ytest))


# Tests

I ran tests over all subsets for synthetic and diabetes data to determine whether or not the "ideal subset" actually performed better predictions. I found that expecially at lower subset sizes, like 2-4 it makes a huge difference. Specifically with the diabetes data, it was basically able to match a model trained on all features with only the features glucose level and age available.

In [ ]:
from tests import test_subsets

model_to_test = RandomForrest
needs_input = False

# Note needs_inputsize should be true for LogisticRegression and NeuralNetwork
for i in range(len(df.columns) - 1):
    naive, rand, ideal = test_subsets(model_to_test, current_graph, n=i, needs_inputsize=True)
    print(str(i) + ":")
    print("Naive:", naive)
    print("Random:", rand)
    print("Ideal:", ideal)

In [ ]:
from tests import test_subsets

models = [LogisticRegression, DecisionTree, BoostedDecisionTree, BaggedDecisionTree, RandomForrest, NeuralNetwork]
needs_input = [True, False, False, False, False, True]
subset_size = 3

# Note needs_inputsize should be true for LogisticRegression and NeuralNetwork
for mod, needs in zip(models, needs_input):
    naive, rand, ideal = test_subsets(mod, current_graph, n=subset_size, needs_inputsize=needs)
    print(str(mod) + ":")
    print("Naive:", naive)
    print("Random:", rand)
    print("Ideal:", ideal)